In [1]:
using Revise
using Pkg; Pkg.activate(".")

using Dates
using Statistics # limited set of methods included in Julia Base
using StatsBase  # extends Statistics with more functions
using Rotations
# using SignalAlignment
using Interpolations
using PyPlot
using JLD2
# using MAT

include("./readers.jl")
using .ShipPosmv

include("./read_lidar.jl")
using .read_vecnav: read_vecnav_dict

include("./timing_lidar.jl")
using .timing_lidar

  Activating project at `~/Projects/ASTRAL/lidar`


In [3]:
# rotations example

# RotXY(roll, pitch) # radians
# matrix that rotates around y (pitch), then by x (roll) in radians
RotXY(10/180*pi, 5/180*pi)

3×3 RotXY{Float64} with indices SOneTo(3)×SOneTo(3)(0.174533, 0.0872665):
  0.996195   0.0        0.0871557
  0.0151344  0.984808  -0.172987
 -0.0858317  0.173648   0.98106

Use `RotXY(ϕ, θ)` when there is no stabilization; i.e., the lidar is strapped down to the deck, as in the first part of EKAMSAT leg 2.


In the second part of EKAMSAT leg 2, the stabilization was working only for the roll axis.
The pitch axis was strapped, so we can synchronize using the pitch angle as in `vectornav.ipynb`.
To start, assume the pitch gimbal is strapped level to the ship deck (there might be a small offset angle).
If the roll stabilization works well, then the lidar coordinate system is rotated only by the pitch angle about the y axis.
Use `RotY(θ)` for this rotation matrix.

For leg 1 we assume it's fully stabilized, so we can measure the difference between the heave at the lidar and at the POSMV.
The heave measured by the POSMV is assumed to be vertical, and the heave at the VectorNav on the lidar, is also vertical when stabilized.
In this case, the displacement moments $(L_x, L_y, L_z)$ between the lidar and the POSMV are responsible for the difference in heave vertical velocity at the lidar $w$ compared to the POSMV $w_0$,
$$
w' = w - w_0 = A\theta + B\phi,\\
A = (L_x^2 + L_z^2)^{1/2},\\
B = (L_y^2 + L_z^2)^{1/2}
$$
where $\theta$ is the pitch and $\phi$ is the roll. This is a regression problem for ${\bf a} = [A; B]$, given $w'$ and $v=[\theta; \phi]$.

$$
{\bf w'} = {\bf av}\\
{\bf a} = {\bf w'}{\bf v}^{-1} = {\bf v}\backslash {\bf w'}.
$$

The rotations and angular rates are each in different coordinate systems. 
It is convenient to add velocity vectors in the pitched (but not rolled) coordinate system, denoted by a prime ($'$).
The lidar is in pitched and rolled coordinate system denoted by double-prime ($''$).
The velocity vector in the pitched coordinate is
$$
{\bf u}' =  {\bf u}_p' + {\bf u}_r' + {\bf w}'_0,\\
{\bf w}'_0 = {\bf R}_\theta{\bf w}_0 = {\bf R}_\theta(0,0,w_0),\\
{\bf u}_p' = {\bf R}_\theta {\bf u}_p = {\bf R}_\theta\dot{\theta}(-L_z,0,L_x),\\
{\bf u}_r' = \dot\phi(0, L_z,-L_y).
$$
In the lidar coordinate system,
$$
{\bf u}'' = {\bf R}_\phi{\bf u}'.
$$
When strapped down, the VectorNav measures one component of ${\bf u}''$, nominally the component in the upward $z$-direction.


In leg 2 part 2 the pitch stabilization gimbal failed
and was strapped down, but the roll stabilization continued. 
For this time, we synchronize the POSMV and VectorNav 
time using the unstabilized pitch angle, and then regress for coefficients
that explain the observed platform velocity in the lidar frame from
rotations about the POSMV.


### Procedure
1. Read POSMV and VectorNav data.
2. Synchronize time in 2-10 min windows.
    - using POSMV pitch and VectorNav roll for leg 2, part 2.
3. Regress pitch and roll angles on heave to get moments $L_x, Ly, Lz$.

In [ ]:
# differentiate POSMV angles to get angular rates

"derivative by FFT, assuming sampling rate of 1."
function fft_derivative(f::AbstractVector{Real})
    N = length(f) 
    k = fftfreq( N ) * 2π  # compute the angular frequencies (wavenumbers)
    # for even N exclude Nyquist frequency according to SG Johnson
    # https://math.mit.edu/~stevenj/fft-deriv.pdf
    if iseven(N)
        k[N/2] = 0
    end
    F_deriv = 1im * k .* fft(f)    # multiply FFT by i*k in the frequency domain
    return real( ifft(F_deriv) )     # inverse FFT to get the derivative in time domain
end

"the angle- and angular-rate dependent coefficient vector"
function w_angles(θ, ϕ, θrate=fft_derivative(θ)/dt, ϕrate=fft_derivative(ϕ)/dt)
    
    return [blah, blah, blah] # vector
end

In [1]:
# read data
Vn = read_vecnav_dict()
Pashr = Dict(Symbol(key) => value for (key, value) in load("./data/table/ASTraL_POSMV.jld2"))

In [ ]:
# deprecated

w0 = Pashr[:heave] # sign?
# heave +down for VectorNav
# need +down right-handed coordinate system for rotations

a = 
v = [ϕ, θ]
# then the heave in the lidar coordinate system is
w_lidar = RotXY(v...)*[0, 0, w0 + a*v]



# then the heave in the lidar coordinate system is
w_lidar = RotY(θ)*( w0 + a*v )